In [ ]:
import numpy as np
np.seterr(invalid='ignore')

import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams.update({'font.size': 12})

import pandas as pd

from my_functions import *

import glob

from astropy.cosmology import Planck18 as cosmo
import astropy.units as u

In [ ]:
w_central = central_wavelength()
nb_fwhm_Arr = nb_fwhm(range(60))
w_lya = 1215.67
filter_tags = load_filter_tags()

In [ ]:
## Load my QSO catalog

filename = '/home/alberto/almacen/Source_cats/QSO_100000_0/'
files = glob.glob(filename + 'data*')
files.sort()
fi = []

for name in files:
    fi.append(pd.read_csv(name))

data_qso = pd.concat(fi, axis=0, ignore_index=True)

qso_flx = data_qso.to_numpy()[:, 1 : 60 + 1].T
qso_err = data_qso.to_numpy()[:, 60 + 1 : 120 + 1].T

qso_flx += qso_err * np.random.normal(size=qso_err.shape)

EW_qso = data_qso['EW0'].to_numpy()
qso_zspec = data_qso['z'].to_numpy()

In [ ]:
## Load my GAL catalog

filename = '/home/alberto/almacen/Source_cats/GAL_100000_0/'
files = glob.glob(filename +'data*')
files.sort()
fi = []

for name in files:
    fi.append(pd.read_csv(name))

data_gal = pd.concat(fi, axis=0, ignore_index=True)

gal_flx = data_gal.to_numpy()[:, 1 : 60 + 1].T
gal_err = data_gal.to_numpy()[:, 60 + 1 : 120 + 1].T

gal_flx += gal_err * np.random.normal(size=gal_err.shape)

EW_gal = np.zeros(data_gal['z'].to_numpy().shape)
gal_zspec = data_gal['z'].to_numpy()

In [ ]:
## Load SF catalog

filename = '/home/alberto/almacen/Source_cats/LAE_10deg_z2-4.25_0/'
files = glob.glob(filename +'data*')
files.sort()
fi = []

for i, name in enumerate(files):
    if i == 10:
        break
    fi.append(pd.read_csv(name))

data = pd.concat(fi, axis=0, ignore_index=True)

sf_flx = data.to_numpy()[:, 1 : 60 + 1].T
sf_err = data.to_numpy()[:, 60 + 1 : 120 + 1].T

sf_flx += sf_err * np.random.normal(size=sf_err.shape)

EW_sf = data['EW0'].to_numpy()
sf_zspec = data['z'].to_numpy()

In [ ]:
pm_flx = np.hstack((qso_flx, sf_flx, gal_flx))
pm_err = np.hstack((qso_err, sf_err, gal_err))
zspec = np.concatenate((qso_zspec, sf_zspec, gal_zspec))
EW_lya = np.concatenate((EW_qso, EW_sf, EW_gal))

N_sf = sf_flx.shape[1]
N_qso = qso_flx.shape[1]
N_gal = gal_flx.shape[1]

qso_dL = cosmo.luminosity_distance(qso_zspec).to(u.cm).value
sf_dL = cosmo.luminosity_distance(sf_zspec).to(u.cm).value

sf_L = data['L_lya'].to_numpy()
qso_L = data_qso['L_lya'].to_numpy()
gal_L = np.zeros(EW_gal.shape)

sf_flambda = 10 ** sf_L / (4*np.pi * sf_dL **2)
qso_flambda = data_qso['F_line']
gal_flambda = np.zeros(EW_gal.shape)

L_lya = np.concatenate((qso_L, sf_L, gal_L))
fline = np.concatenate((qso_flambda, sf_flambda, gal_flambda))

is_qso = np.concatenate((np.ones(N_qso), np.zeros(N_sf + N_gal))).astype(bool)
is_sf = np.concatenate((np.zeros(N_qso), np.ones(N_sf), np.zeros(N_gal))).astype(bool)
is_gal = np.concatenate((np.zeros(N_qso), np.zeros(N_sf), np.ones(N_gal))).astype(bool)

%xdel sf_flx
%xdel sf_err
%xdel qso_flx
%xdel qso_err
%xdel gal_flx
%xdel gal_err
%xdel sf_zspec
%xdel qso_zspec
%xdel gal_zspec
%xdel EW_sf
%xdel EW_qso
%xdel EW_gal
%xdel qso_dL
%xdel sf_L
%xdel qso_L
%xdel sf_flambda
%xdel qso_flambda
%xdel gal_flambda
%xdel data
%xdel data_qso
%xdel data_gal

In [ ]:
mag = flux_to_mag(pm_flx[-2], w_central[-2])
mag[np.isnan(mag)] = 99.

N_sources = pm_flx.shape[1]
N_sources

In [ ]:
ew0_cut = 20
# Lya search
cont_est_lya, cont_err_lya = estimate_continuum(pm_flx, pm_err, IGM_T_correct=True)
line = is_there_line(pm_flx, pm_err, cont_est_lya, cont_err_lya, ew0_cut)
lya_lines, lya_cont_lines, line_widths = identify_lines(
    line, pm_flx, cont_est_lya, first=True, return_line_width=True
)
lya_lines = np.array(lya_lines)

# Other lines
cont_est_other, cont_err_other = estimate_continuum(pm_flx, pm_err, IGM_T_correct=False)
line_other = is_there_line(pm_flx, pm_err, cont_est_other, cont_err_other,
    400, obs=True)
other_lines = identify_lines(line_other, cont_est_other, pm_err)

# Compute z
z_Arr = np.zeros(N_sources)
z_Arr[np.where(np.array(lya_lines) != -1)] =\
    z_NB(np.array(lya_cont_lines)[np.where(np.array(lya_lines) != -1)])

nice_z = np.abs(z_Arr - zspec) < 0.16

%xdel cont_est_other
%xdel cont_err_other

In [ ]:
mag_min = 17
mag_max = 24

nb_min = 5
nb_max = 15
# nb_min = 16
# nb_max = 23

# Used later!!
L_min = 40
L_max = 50

nbs_to_consider = np.arange(nb_min, nb_max + 1)

nb_cut = (np.array(lya_lines) >= nb_min) & (np.array(lya_lines) <= nb_max)

z_min = (w_central[nb_min] - nb_fwhm_Arr[nb_min] * 0.5) / w_lya - 1
z_max = (w_central[nb_max] + nb_fwhm_Arr[nb_max] * 0.5) / w_lya - 1
print(f'z interval: ({z_min:0.2f}, {z_max:0.2f})')

z_cut = (z_min < z_Arr) & (z_Arr < z_max)
zspec_cut = (z_min < zspec) & (zspec < z_max)
ew_cut = EW_lya > ew0_cut
mag_cut = (mag > mag_min) & (mag < mag_max)

nice_lya = nice_lya_select(
    lya_lines, other_lines, pm_flx, pm_err, cont_est_lya, z_Arr
)
nice_lya = (nice_lya & z_cut & mag_cut)

In [ ]:
EW_nb_Arr, EW_nb_e, L_Arr, L_e_Arr, flambda, flambda_e = EW_L_NB(
    pm_flx, pm_err, cont_est_lya, cont_err_lya, z_Arr, lya_lines, N_nb=0
)

ML_predict_mask = (mag < 23) & (L_Arr > 0)
L_Arr[ML_predict_mask] = ML_predict_L(
    pm_flx[:, ML_predict_mask], pm_err[:, ML_predict_mask],
    z_Arr[ML_predict_mask], L_Arr[ML_predict_mask], 'RFmag15-23'
)

ML_predict_mask = (mag > 23) & (L_Arr > 0)
L_Arr[ML_predict_mask] = ML_predict_L(
    pm_flx[:, ML_predict_mask], pm_err[:, ML_predict_mask],
    z_Arr[ML_predict_mask], L_Arr[ML_predict_mask], 'RFmag23-23.5'
)

In [ ]:
selected = np.random.permutation(
    np.where(
        nice_lya & (L_Arr > 44) & (zspec > z_min) & (zspec < z_max) & mag_cut & ~nice_z
    )[0]
)
print(count_true(selected))
print(z_min, z_max)

for i, src in enumerate(selected):
    if i == 10: break
    print(is_qso[src], is_sf[src])
    print(L_lya[src])
    print(L_Arr[src])

    lya_obs_w = w_central[lya_lines[src]]
    other_lines_w = [w_central[i] for i in other_lines[src]]

    fig = plt.figure(figsize=(10, 5))
    ax = plot_JPAS_source(pm_flx[:, src], pm_err[:, src])

    ax.axvline(lya_obs_w, linestyle='--', color='r')
    for w in other_lines_w:
        ax.axvline(w, linestyle='--', color='dimgray')

    ax.set_title(f'zspec={zspec[src]:0.3f}, z_Arr={z_Arr[src]:0.3f}')

    plt.show()

In [ ]:
plt.hist(zspec[(44 < L_lya) & (L_lya < 45)], bins=np.linspace(2, 4.5, 20))

plt.show()